In [ ]:
import os; os.environ['GEMINI_API_KEY'] = 'AIzaSyB7DEYe_bQ__xxxx'

In [ ]:
!pip install google-genai

In [ ]:
!wget -O image.jpg https://www.shutterstock.com/image-vector/coloring-page-outline-cartoon-smiling-600nw-2037524801.jpg

--2025-03-17 20:21:14--  https://www.shutterstock.com/image-vector/coloring-page-outline-cartoon-smiling-600nw-2037524801.jpg
Resolving www.shutterstock.com (www.shutterstock.com)... 18.154.101.39, 18.154.101.115, 18.154.101.119, ...
Connecting to www.shutterstock.com (www.shutterstock.com)|18.154.101.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56269 (55K) [image/jpeg]
Saving to: ‘image.jpg’

image.jpg           100%[===================>]  54.95K  --.-KB/s    in 0.01s   

2025-03-17 20:21:14 (4.32 MB/s) - ‘image.jpg’ saved [56269/56269]



In [ ]:
import base64
import os
from google import genai
from google.genai import types


def save_binary_file(file_name, data):
    f = open(file_name, "wb")
    f.write(data)
    f.close()


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    files = [
        # Make the file available in local system working directory
        client.files.upload(file="image.jpg"),
    ]
    model = "gemini-2.0-flash-exp-image-generation"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=files[0].uri,
                    mime_type=files[0].mime_type,
                ),
                types.Part.from_text(text="""please color this beautifully using red, green, blue"""),
            ],
        )
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        top_k=40,
        max_output_tokens=8192,
        response_modalities=[
            "image",
            "text",
        ],
        safety_settings=[
            types.SafetySetting(
                category="HARM_CATEGORY_CIVIC_INTEGRITY",
                threshold="OFF",  # Off
            ),
        ],
        response_mime_type="text/plain",
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if not chunk.candidates or not chunk.candidates[0].content or not chunk.candidates[0].content.parts:
            continue
        if chunk.candidates[0].content.parts[0].inline_data:
            file_name = "output.jpg"
            save_binary_file(
                file_name, chunk.candidates[0].content.parts[0].inline_data.data
            )
            print(
                "File of mime type"
                f" {chunk.candidates[0].content.parts[0].inline_data.mime_type} saved"
                f"to: {file_name}"
            )
        else:
            print(chunk.text)

if __name__ == "__main__":
    generate()

File of mime type image/png savedto: output.jpg
